In [ ]:
import the necessary packages from imutils import paths import numpy as np
import matplotlib.pyplot as plt import glob
import math import argparse import imutils import cv2


#IMAGE STITCHING

# LOAD IMAGES

print("[INFO] loading images...") imagefiles = glob.glob("1.jpg/*") imagefiargp
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) images.append(img)
num_images = len(images)

#Display images plt.figure(figsize=[30,10]) num_cols = 3
num_rows = math.ceil(num_images / num_cols) for i in range(0, num_images):
plt.subplot(num_rows, num_cols, i+1) plt.axis('off')
plt.imshow(images[i])


#STITCHING IMAGES

print("[INFO] stitching images...") stitcher = cv2.Stitcher_create()
(status, stitched) = stitcher.stitch(images)

stitched = cv2.cvtColor(stitched, cv2.COLOR_BGR2RGB) cv2.imwrite("./res/result.png",stitched)
# if the status is '0', then OpenCV successfully performed image stitching if status == 0:
# display the output stitched image to our screen plt.figure(figsize=[30,10])
plt.imshow(stitched) cv2.waitKey(0)
# otherwise the stitching failed, likely due to not enough keypoints being detected else:
print("[INFO] image stitching failed ({})".format(status))

#CROPPING IMAGE

print("[INFO] cropping...")

stitched = cv2.copyMakeBorder(stitched, 10, 10, 10, 10, cv2.BORDER_CONSTANT,

(0, 0, 0))


# convert the stitched image to grayscale and threshold it # such that all pixels greater than zero are set to 255
# (foreground) while all others remain 0 (background) gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY) plt.imshow(thresh)
cv2.waitKey(0)


# find all external contours in the threshold image then find

# the *largest* contour which will be the contour/outline of the stitched image cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
cv2.CHAIN_APPROX_SIMPLE)

cnts = imutils.grab_contours(cnts)

c = max(cnts, key=cv2.contourArea)

# allocate memory for the mask which will contain the rectangular bounding box of the stitched image region
mask = np.zeros(thresh.shape, dtype="uint8") (x, y, w, h) = cv2.boundingRect(c)
cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

# create two copies of the mask: one to serve as our actual

# minimum rectangular region and another to serve as a counter

# for how many pixels need to be removed to form the minimum rectangular region minRect = mask.copy()
sub = mask.copy()


# keep looping until there are no non-zero pixels left in the subtracted image while cv2.countNonZero(sub) > 0:
# erode the minimum rectangular mask and then subtract

# the thresholded image from the minimum rectangular mask # so we can count if there are any non-zero pixels left
minRect = cv2.erode(minRect, None) sub = cv2.subtract(minRect, thresh)


# find contours in the minimum rectangular mask and then # extract the bounding box (x, y)-coordinates
cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

c = max(cnts, key=cv2.contourArea) (x, y, w, h) = cv2.boundingRect(c)


plt.imshow(minRect) cv2.waitKey(0)
# use the bounding box coordinates to extract the our final

# stitched image

stitched = stitched[y:y + h, x:x + w]

# write the output stitched image to disk cv2.imwrite("./res/final.png", stitched)
# display the output stitched image to our screen plt.imshow(stitched)
cv2.waitKey(0)


#TEXTURE SEGMENTATION

from skimage import io

from skimage.filters import threshold_otsu


img=io.imread("./res/final.png", as_gray=True) plt.imshow(img,cmap="gray")


#threshold=filters.threshold_niblack(img) from scipy import ndimage
k=20

img_mean=ndimage.uniform_filter(img,(k,k)) img_sqr_mean=ndimage.uniform_filter(img**2,(k,k)) img_var=img_sqr_mean - img_mean**2 plt.imshow(img_var,cmap='gray')

ksize=45 theta=np.pi/2

kernel=cv2.getGaborKernel((ksize,ksize), 5.0, theta, 10.0, 0.9, 0, ktype=cv2.CV_32F) filtered_image=cv2.filter2D(img, cv2.CV_8UC3, kernel)
plt.imshow(filtered_image, cmap='gray')


from skimage.filters.rank import entropy from skimage.morphology import disk


entropy_img =entropy(img,disk(5)) plt.imshow(entropy_img)


plt.hist(entropy_img.flat,bins=100, range=(0,6)) thresh= threshold_otsu(entropy_img)
binary = entropy_img<=thresh plt.imshow(binary)
